In [4]:
# ✅ KoSentenceBERT 기반 의미 유사도 평가 코드
# 설치 (최초 1회만 필요)
%pip install sentence-transformers pandas

import pandas as pd
from sentence_transformers import SentenceTransformer, util


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
# ✅ 1. 모델 로드 (한국어 SBERT)
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd

# Mac GPU (MPS) 사용 설정
if torch.backends.mps.is_available():
    device = 'mps'
    print("🍎 Mac GPU (MPS)를 사용합니다")
elif torch.cuda.is_available():
    device = 'cuda'
    print("🚀 NVIDIA GPU (CUDA)를 사용합니다")
else:
    device = 'cpu'
    print("💻 CPU를 사용합니다")

print(f"선택된 디바이스: {device}")

# 모델 로드 및 디바이스 이동
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

try:
    model = model.to(device)
    print(f"✅ 모델을 {device}로 이동완료")
except Exception as e:
    print(f"⚠️ {device} 사용 중 오류 발생: {e}")
    device = 'cpu'
    model = model.to(device)
    print("🔄 CPU로 fallback")

# ✅ 2. 데이터 불러오기
merged_df = pd.read_csv("merged_news_data (3).csv")
original_df = pd.read_csv("NewsResult_2025뉴스.xlsx - sheet.csv")

print(f"merged_df 크기: {merged_df.shape}")
print(f"original_df 크기: {original_df.shape}")


🍎 Mac GPU (MPS)를 사용합니다
선택된 디바이스: mps
✅ 모델을 mps로 이동완료
merged_df 크기: (10952, 9)
original_df 크기: (10952, 19)


In [16]:
# ✅ 3. 데이터 병합 및 준비
data = pd.merge(merged_df, original_df, on='뉴스 식별자')
summaries = data['요약문'].astype(str).tolist()
originals = data['본문'].astype(str).tolist()

print(f"병합된 데이터 크기: {data.shape}")
print(f"요약문 개수: {len(summaries)}, 원문 개수: {len(originals)}")

# 첫 번째 요약문과 원문 예시 확인
print("\n첫 번째 요약문 예시:")
print(summaries[0][:200] + "..." if len(summaries[0]) > 200 else summaries[0])
print("\n첫 번째 원문 예시:")
print(originals[0][:200] + "..." if len(originals[0]) > 200 else originals[0])

병합된 데이터 크기: (271748, 27)
요약문 개수: 271748, 원문 개수: 271748

첫 번째 요약문 예시:
이재명 대통령은 경기도 가평과 충남 서산·예산, 전남 담양, 경남 산청·합천 등 6개 시군을 특별재난지역으로 선포했는데, 엄혹한 현장에서 음주가무를 즐기는 공직자들의 처신 문제도 거론했다.

첫 번째 원문 예시:
[앵커]

 이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했습니다.

 이 대통령은 재난 상황에서 부적절한 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문했습니다.

 보도에 이희연 기자입니다.

 [리포트]

경기도 가평과 충남 서산 예산..


In [ ]:
# ✅ 5. 임베딩 및 유사도 평가
from tqdm import tqdm
import torch
from sentence_transformers import util

def get_memory_info():
    """현재 디바이스에 맞는 메모리 사용량 정보 반환"""
    if device == 'mps':
        try:
            # MPS는 메모리 정보 조회가 제한적
            return "🍎 MPS 사용 중 (메모리 정보 제한적)"
        except:
            return "🍎 MPS 메모리 정보 조회 불가"
    elif device == 'cuda':
        try:
            allocated = torch.cuda.memory_allocated() / 1024**3
            cached = torch.cuda.memory_reserved() / 1024**3
            return f"🚀 CUDA 메모리 - 할당: {allocated:.2f}GB, 캐시: {cached:.2f}GB"
        except:
            return "🚀 CUDA 메모리 정보 조회 불가"
    else:
        return "💻 CPU 모드"

def encode_with_progress(model, texts, batch_size=32, description="Encoding"):
    embeddings = []
    print(f"시작 전: {get_memory_info()}")
    
    for i in tqdm(range(0, len(texts), batch_size), desc=description):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
        embeddings.append(batch_embeddings)
        
        # 메모리 정리 (디바이스별 처리)
        if i % (batch_size * 10) == 0:
            if device == 'cuda':
                torch.cuda.empty_cache()
            elif device == 'mps':
                torch.mps.empty_cache()  # MPS 캐시 정리
    
    result = torch.cat(embeddings, dim=0)
    print(f"완료 후: {get_memory_info()}")
    return result

# 시작 전 메모리 상태
print("🔄 임베딩 생성 시작...")
print(f"시작 시점: {get_memory_info()}")

# 임베딩 생성
print("\n1. 요약문 임베딩 중...")
summary_emb = encode_with_progress(model, summaries, batch_size=32, description="요약문")

print("\n2. 원문 임베딩 중...")
original_emb = encode_with_progress(model, originals, batch_size=32, description="원문")

# 유사도 계산
print("\n3. 유사도 계산 중...")
similarities = util.cos_sim(summary_emb, original_emb).diagonal()
score = similarities.mean().item()

# 결과 출력
print(f"\n✅ 평가 결과:")
print(f"평균 유사도: {score:.4f}")
print(f"최고 유사도: {similarities.max().item():.4f}")
print(f"최저 유사도: {similarities.min().item():.4f}")
print(f"표준편차: {similarities.std().item():.4f}")

# 결과를 데이터에 추가
if USE_SAMPLE:
    sample_data['KoSBERT_Score'] = similarities.cpu().numpy()
    print(f"\n상위 5개 기사 점수:")
    print(sample_data[['뉴스 식별자', 'KoSBERT_Score']].head())
else:
    data['KoSBERT_Score'] = similarities.cpu().numpy()
    print(f"\n상위 5개 기사 점수:")
    print(data[['뉴스 식별자', 'KoSBERT_Score']].head())

print(f"\n최종: {get_memory_info()}")

🔄 임베딩 생성 시작...
시작 시점: 🍎 MPS 사용 중 (메모리 정보 제한적)

1. 요약문 임베딩 중...
시작 전: 🍎 MPS 사용 중 (메모리 정보 제한적)


요약문:   2%|▏         | 134/8493 [00:29<29:03,  4.79it/s]

In [ ]:
# ✅ 4. 샘플링 옵션 (빠른 테스트용)
USE_SAMPLE = True  # 빠른 테스트를 위해 True로 설정
SAMPLE_SIZE = 1000

if USE_SAMPLE:
    import numpy as np
    sample_indices = np.random.choice(len(data), min(SAMPLE_SIZE, len(data)), replace=False)
    sample_data = data.iloc[sample_indices].copy()
    summaries = sample_data['요약문'].astype(str).tolist()
    originals = sample_data['본문'].astype(str).tolist()
    print(f"🚀 샘플 모드: {len(summaries)}개 데이터로 테스트")
else:
    print(f"📊 전체 데이터 모드: {len(summaries)}개 데이터로 평가")

# 현재 설정 확인
print(f"\n🔧 디바이스 정보:")
print(f"- 모델 디바이스: {model.device}")
print(f"- 현재 사용 디바이스: {device}")

# Mac MPS 정보
if torch.backends.mps.is_available():
    print("🍎 Mac MPS (Metal Performance Shaders) 지원")
    print(f"- MPS 사용 가능: {torch.backends.mps.is_built()}")
else:
    print("❌ MPS 지원 안됨")

# CUDA 정보 (Windows/Linux)
if torch.cuda.is_available():
    try:
        device_count = torch.cuda.device_count()
        current_device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_device)
        
        print(f"🚀 CUDA 정보:")
        print(f"- GPU 개수: {device_count}")
        print(f"- 현재 GPU: {device_name}")
        
        total_memory = torch.cuda.get_device_properties(current_device).total_memory
        allocated = torch.cuda.memory_allocated(current_device)
        cached = torch.cuda.memory_reserved(current_device)
        
        print(f"- 총 GPU 메모리: {total_memory / 1024**3:.1f}GB")
        print(f"- 할당된 메모리: {allocated / 1024**3:.2f}GB")
        print(f"- 캐시된 메모리: {cached / 1024**3:.2f}GB")
        
    except Exception as e:
        print(f"- CUDA 정보 조회 중 오류: {e}")

# CPU 정보
if device == 'cpu':
    print("💻 CPU 모드로 실행됩니다")
    
print(f"\n📊 처리할 데이터 개수: {len(summaries)}")

NameError: name 'summaries' is not defined